In [ ]:
from PIL import Image
import pytesseract
from scipy.ndimage import interpolation as inter

%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display

#import cv2

import os
import collections

from os import listdir
from os.path import isfile, join
from difflib import SequenceMatcher

import pandas as pd
import re

import random

In [ ]:
scenario = ["name", "date", "address"]
cropped_img_path = 'cropped_v2/cropped_v2/'


In [ ]:
s = scenario[1]
onlyfiles = [f for f in listdir(cropped_img_path+s+"/") if isfile(join(cropped_img_path+s+"/", f))]
onlyfiles[:5]

In [ ]:
tha_months = ["ม.ค.", "มี.ค.", "เม.ย.", "พ.ค.", "มิ.ย.", "ก.ค.", "ส.ค.", "ก.ย.", "ต.ค.", "พ.ย.", "ธ.ค."]
eng_months = ["Jan.", "Feb.", "Mar.", "Apr.", "May.", "Jun.", "Jul.", "Aug.", "Sep.", "Oct.", "Nov.", "Dec."]

In [ ]:
def eng_month_matching(eng_months, text,eng_month_ratio_threshold):
    """Match English month"""    
    # calculate similarity between candidate and titles
    month = [(t, similar(text.lower(), t.replace(".","").lower())) for t in eng_months]
    # sort by the highest similarity
    max_ratio_month = sorted(month, key=lambda tup: -tup[1])
    print(max_ratio_month)
    # check if ratio is higher then min
    if max_ratio_month[0][1] >= eng_month_ratio_threshold:
        result_month = max_ratio_month[0][0]
    else:
        result_month = ""
    return result_month


def clean_date(text):
    """Remove punctuations from extracted name"""
    # replace some punctuations by space
    text = text.replace("/", " ").replace("\"", " ").replace("%", "3")
    # remove punctuation
    punctuations = "!@#$%^&*()_+=,.-/<>\|™;~:©®\[\]\{\}“\"\‘\'\’"
    cleaned_text = ''.join(ch for ch in text if ch not in punctuations)
    return cleaned_text

def choose_higher(value1, freq1, value2, freq2):
    if freq1 >= freq2:
        if freq1 > 4:
            value = value1
        else: 
            value = value2
    else:
        value = value2
    return value

def extract_date_modes_balancing(img, step,lang,config_p_eng_8,config_p_eng_12, ranges, date_type='bd'):
    # extract text with different modes
    day_month1, day_month_freq1, mil1, mil_freq1, cen1, cen_freq1, ten1, ten_freq1, yr1, yr_freq1 = extract_date(img, step,lang,config_p_eng_12, ranges, date_type)
    day_month2, day_month_freq2, mil2, mil_freq2, cen2, cen_freq2, ten2, ten_freq2, yr2, yr_freq2 = extract_date(img, step,lang,config_p_eng_8, ranges, date_type)

    day_month = choose_higher(day_month1, day_month_freq1, day_month2, day_month_freq2)
    mil = choose_higher(mil1, mil_freq1, mil2, mil_freq2)
    cen = choose_higher(cen1, cen_freq1, cen2, cen_freq2)
    ten = choose_higher(ten1, ten_freq1, ten2, ten_freq2)
    yr = choose_higher(yr1, yr_freq1, yr2, yr_freq2)
        
    return day_month + ' ' + mil + cen + ten + yr

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def update_freq(value, elems, max_elem):
    if value in elems:
        elems[value] += 1
    else:
        elems[value] = 1
    return max_elem if max_elem[1] > elems[value] else (value, elems[value])

def extract_date(img, step,  lang,config_p, ranges, date_type):
    # get image size
    w, h = img.size
    print("Size", (w,h))
    # define variables
    freqs = {}
    max_freq = ("", 0)
    tens = {}
    years = {}
    centuries = {}
    millenia = {}
    max_ten = ('', 0)
    max_yr = ('', 0)
    max_cen = ('', 0)
    max_mil = ('', 0)
    total_count = 0
    first_part = '19'
    correlation_cen = {'0' : '0', '1' : '9', '2' : '9', '3' : '9', '4' : '9', '5' : '9', '6' : '0', '7' : '9', '8' : '0', '9' : '9'}
    correlation_mil = {'0' : '0', '1' : '1', '2' : '2', '3' : '2', '4' : '1', '5' : '2', '6' : '1', '7' : '1', '8' : '2', '9' : '2'}
    
    # iterate through size
    for i in ranges:
        # size values
        wb, hb = int((1 + i*step) * w), int((1 + i*step) * h)
        
        # text extraction
        txt = pytesseract.image_to_string(img.resize((wb, hb)), lang, config=config_p)
        
        # count frequency only last word
        txt_last=txt
        # find the most frequent result
        if txt_last != "" and not "\n" in txt_last:
            parts = txt_last.split()
            index = -1
            year = ''.join(c for c in parts[index] if c.isdigit())
            while (index != -len(parts)) and (not year.isdigit()) and (len(year) != 4):
                index -= 1
                year = ''.join(c for c in parts[index] if c.isdigit())
            if index == -len(parts):
                index += 1
                txt_last = ' '.join(parts[:-1])
                year = ''.join(c for c in parts[-1] if c.isdigit())
            else:
                txt_last = ' '.join(parts[:index])
            if year.isdigit() and len(year) == 4:
                total_count += 1
                mil = year[0]
                cen = year[1]
                ten = year[2]
                yr = year[3]
                max_cen = update_freq(cen, centuries, max_cen)
                max_mil = update_freq(mil, millenia, max_mil)
                max_yr = update_freq(yr, years, max_yr)
                max_ten = update_freq(ten, tens, max_ten)
                
                if txt_last in freqs:
                    freqs[txt_last] += 1
                else:
                    freqs[txt_last] = 1
            else:
                if txt_last not in freqs:
                    freqs[txt_last] = 0

            max_freq = max_freq if max_freq[1] > freqs[txt_last] else (txt_last, freqs[txt_last])
        
            print(txt)
            
    final_first_part = ''

    final_mil = correlation_mil[max_mil[0]] if max_mil[0] != '' else ('1' if date_type == 'bd' else '2')
    final_cen = correlation_cen[max_cen[0]] if max_cen[0] != '' else ('9' if date_type == 'bd' else '0')
    
    if (final_mil == '1' and final_cen == '0') or (final_mil == '2' and final_cen == '9'):
        if date_type == 'bd':
            final_mil = '1'
            final_cen = '9'
        else:
            final_mil = '2'
            final_cen = '0'
    
    final_first_part = correlation_mil[final_mil] + correlation_cen[final_cen]

    final_ten = max_ten[0]
    final_yr = max_yr[0]
    if final_ten == '':
        final_ten = str(random.randint(0, 9))
    if final_yr == '':
        final_yr = str(random.randint(0, 9))
    result_date = max_freq[0]
    final_year = final_first_part + final_ten + final_yr
    
    if date_type == 'bd' and int(final_year) > 2018 and (max_ten[1] >= float(total_count * 0.7) and max_yr[1] >= float(total_count * 0.7)):
        final_year = '19' + final_ten + final_yr
    
    final_day_month = clean_date(max_freq[0])
    result = final_day_month + ' ' + final_year
    
    print("====================")
    print("Result = ", result)
    
    return final_day_month, max_freq[1], final_year[0], max_mil[1], final_year[1], max_cen[1], final_ten, max_ten[1], final_yr, max_yr[1] 

In [ ]:
def get_month_from_day(text, day,eng_months,eng_month_ratio_threshold):
    "Find the month from date position"
    # find date first and end position
    start_index = text.find(day)
    end_index = start_index + len(day)
    # get month
    month = text[(end_index+1):].split(" ")[0]
    print(month)
    return  eng_month_matching(eng_months, month,eng_month_ratio_threshold)

def bd_year_validator(year):
    if int(year) >= bd_year[0] and int(year) <= bd_year[1]:
        pass
    else:
        if year[1] == '0':
            year = year[0] + '9' + year[2:]
    return year

def find_bd_date(text,bd_date_ratio_threshold,eng_month_ratio_threshold,eng_months):
    """Find the title and name"""
    words = text.split(" ")
    words = [w for w in words if w != ""]
    # standart situation when there are 3 words
    if len(words) == 3: 
        # check if the words Date and Birth exist
        try:
            year = re.match(r'.*([1-3][0-9]{2,3})', text).group(1)
        except: 
            year = re.match(r'\d+', text).group(1)
        try:
            day = re.match(r'([0-9][0-9]{0,1}) ', text).group(1).replace(" ", "")
        except: 
            day = "-"
        try:
            month_text = re.sub("[^a-zA-Z]+", "", text)
            month = eng_month_matching(eng_months, month_text,eng_month_ratio_threshold)
        except:
            month = "-"
    # if there are 5 words 
    elif  len(words) == 1: 
        year = re.match(r'.*([1-3][0-9]{3})', text).group(1)
        month = "-"
        day = " - "        
    # if there are 2 words 
    elif  len(words) == 2: 
        # extract year
        year = re.match(r'.*([1-3][0-9]{3})', text).group(1)  
        # match the date merged with month text
        try: 
            day = re.match(r'([0-9][0-9]{0,1})', text).group(1).replace(" ", "")            
            # subtract day text
            month_text = re.sub("[^a-zA-Z]+", "", text)
            month = eng_month_matching(eng_months, month_text,eng_month_ratio_threshold)
        except:
            day = "-"            
            month = eng_month_matching(eng_months, text,eng_month_ratio_threshold)
    else:
        try:
            day =  re.match(r'([0-9][0-9]{0,1}) ', text).group(1).replace(" ", "")
        except:
            day = "-"
        try:
            month = get_month_from_day(text, day,eng_months,eng_month_ratio_threshold)
        except:
            month = "-"
        try:
            year = re.match(r'.*([1-3][0-9]{3})', text).group(1)
        except:
            year = re.match(r'\d+', text).group(1)
    return day, month, year

In [ ]:
fields = dict()
fields["name"] = "name_eng"
fields["name"] = "last_name_eng"
fields["name"] = "name_tha"

config_p_eng_12="--psm 12 --oem 1"
config_p_eng_8="--psm 8 --oem 1"

config_p_tha="--psm 4 --oem 0"

step = 0.2
ranges=range(20, 50)
ranges_tha = range(25,35)
ranges_date_eng = range(5,50)

name_ratio_threshold=0.25
last_name_ratio_threshold=0.25
title_ratio_threshold=0.3
tha_name_ratio_threshold=0.15
tha_title_ratio_threshold=0.25
bd_date_ratio_threshold = 0.45
eng_month_ratio_threshold=0.15

bd_year = (1900, 2020)

In [ ]:
file_type = "bd_eng"
bd_files = [x for x in onlyfiles if file_type in x]

date_type = 'bd' if file_type == 'bd_eng' else 'ex'

# read the image
lang = "eng"
img = Image.open(cropped_img_path +s+"/"+bd_files[0])

# find the best output
text = extract_date_modes_balancing(img, step,lang,config_p_eng_12,config_p_eng_8, ranges_date_eng, date_type)
print("=============")
print("Text = ", text)
    
# split by words
day, month, year =  find_bd_date(text,bd_date_ratio_threshold,eng_month_ratio_threshold,eng_months)
print("========================================")
print("Day = ", day)
print("Month = ", month)
print("Year = ", year)

In [ ]:
img

In [ ]:
text = "17 Aug 1955"
year =re.match(r'.*([1-3][0-9]{2,3})', text).group(1)
year

In [ ]:
cropped_img_path +s+"/"+bd_files[8]

In [ ]:
similar("mar", "mav")

In [ ]:
(w, h) = img.size
print("Image size = ", (w,h))
step = 0.2

for i in range(10, 50):
    print(i,"*",step," = ",(int(w*(i*step)), int(h*(i*step))), \
          " resize =>", \
          pytesseract.image_to_string(img.resize((int(w*(i*step)), int(h*(i*step)))), "tha", config="--psm 5 --oem 1"))